# Examining GNMA II Loan-Level Data

Now that it's all in one big H5 file.

3. How hard will it be to examine all data for a given loan?
1. What are the extant MTA codes? Do we have all of them?
2. Are there records of loans that have refinanced in the past?
4. What do the distributions of the relevant factors look like?

In [53]:
import tables
import pandas as pd
import ggplot
import numpy as np
path = "/data/prepayments/GNM_II_loanlevel.h5"

In [28]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['record']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [35]:
h5file.close()

In [41]:
h5file = tables.open_file(path,"r")

In [42]:
h5file.root.loans[-1]

(0, 550, 1383278400, 0, 679, 0, 0, 1013908161, 1288584000, 1, 3355, 37, 0, 4250.0, 18000, 1, 9650, -2062290496, 0, 0, 1, 360, 29800000, 0, '004833', 1, 0, 322, 0, 0, 'OR', 0, 28190523, 29800000, 2250)

In [5]:
print "  ".join(sorted(h5file.root.loans.coldescrs.keys()))

agency  annual_mip  as_of_date  buy_down_status  credit_score  current_month_liquidation  down_payment_assistance  dsn  first_payment_date  first_time  issuer_id  loan_age  loan_gross_margin  loan_interest_rate  loan_origination_date  loan_purpose  ltv  maturity_date  months_delinquent  months_prepaid  number_of_borrowers  original_loan_term  original_principal_balance  origination_type  pool_id  property_type  refinance_type  remaining_loan_term  removal_reason  seller_issuer_id  state  total_debt_expense_ratio  upb  upb_at_issuance  upfront_mip


# 1. How hard to gather per-loan data?

In [12]:
h5file.root.loans[-1]['dsn']

1507407588

In [13]:
h5file.root.loans[0]['dsn']

1023208624

In [14]:
import datetime

In [30]:
for i,record in enumerate(h5file.root.loans.where('dsn == 1023208624')):
    print "\t".join([str(i), 
                     '{:%Y-%m}'.format(datetime.datetime.fromtimestamp(record['as_of_date'])),
                     str(record['ltv']), str(record['removal_reason']), str(record['loan_age'])])

0	2013-10	9566	0	257


Loan origination date not useful. 

Even with an index, it's still pretty slow to retrieve.

# 2. How do the Metropolitan Statistical Areas shake out?

A. There aren't any.

In [46]:
h5file.root.loans.shape

(7169903,)

274 million loan records.

In [17]:
random_samples = np.random.choice(h5file.root.loans.shape[0],10000)

In [18]:
random_samples[:10]

array([238363026, 239711183, 206973215,  16056807, 128508040,   7554948,
       211847000,  84285540,  38631633, 166621288])

MSAs weren't actually in the data. Should I backfill the states with MSA data? Or is there other data?

# 3. What does a prepaid loan look like?

In [20]:
def dt_to_string(x):
    return '{:%Y-%m}'.format(datetime.datetime.fromtimestamp(x))

In [22]:
import time
last_month = time.mktime(datetime.date(2014,10,1).timetuple())
last_month

1412136000.0

In [45]:
for i,record in enumerate(h5file.root.loans.where("(current_month_liquidation == 0) " +
                                                  "& (as_of_date > {})".format(last_month))):
    if i == 1:
        print "\t".join([str(record[x]) for x in ['dsn','loan_age']]+[dt_to_string(record['as_of_date'])])
        break

What does a negative loan age number mean?

In [43]:
for i,record in enumerate(h5file.root.loans.where('loan_age < 0')):# & (as_of_date < {})'.format(last_month))):
    if i < 10:
        print "\t".join([str(i), 
                     '{:%Y-%m}'.format(datetime.datetime.fromtimestamp(record['as_of_date'])),
                     str(record['as_of_date']),str(record['ltv']),
                     str(record['current_month_liquidation']), str(record['loan_age'])])
    else:
        break

Prepaid loans do *not* stick around in the dataset. So we have a pretty limited universe of prepaid loans, time-wise, unless I can find some data from before 2013-10.

In [47]:
for x in h5file.root.loans.where('(dsn == 1023494824)'):
    print x
    break

/loans.row (Row), pointing to row #2535


In [50]:
zip(h5file.root.loans.colnames,h5file.root.loans[2535])

[('agency', 0),
 ('annual_mip', 500),
 ('as_of_date', 1380600000),
 ('buy_down_status', 0),
 ('credit_score', 0),
 ('current_month_liquidation', 0),
 ('down_payment_assistance', 0),
 ('dsn', 1023494824),
 ('first_payment_date', 786258000),
 ('first_time', 0),
 ('issuer_id', 3355),
 ('loan_age', 227),
 ('loan_gross_margin', 0),
 ('loan_interest_rate', 8500.0),
 ('loan_origination_date', 18000),
 ('loan_purpose', 1),
 ('ltv', 9677),
 ('maturity_date', 1730433600),
 ('months_delinquent', 0),
 ('months_prepaid', 0),
 ('number_of_borrowers', 1),
 ('original_loan_term', 360),
 ('original_principal_balance', 4700000),
 ('origination_type', 0),
 ('pool_id', '001901'),
 ('property_type', 1),
 ('refinance_type', 0),
 ('remaining_loan_term', 133),
 ('removal_reason', 0),
 ('seller_issuer_id', 0),
 ('state', 'TX'),
 ('total_debt_expense_ratio', 0),
 ('upb', 3136686),
 ('upb_at_issuance', 0),
 ('upfront_mip', 2250)]

# What's blank?

Things to check:
* msa
* ~~loan origination date~~
* ~~upb as issuance~~
* ~~Loan gross margin (?) ARM only~~
* Combined LTV
* ~~Total debt expense ratio percent~~
* ~~First Time home buyer~~
* ~~3rd party origination type~~
* ~~removal reason~~

In [33]:
for i, record in enumerate(h5file.root.loans.where('loan_origination_date > 18000')):
    if i > 40:
        break
    else:
        print "\t".join([str(i), 
                     '{:%Y-%m}'.format(datetime.datetime.fromtimestamp(record['as_of_date'])),
                     '{:%Y-%m}'.format(datetime.datetime.fromtimestamp(record['loan_origination_date'])),
                     str(record['loan_origination_date']),
                     str(record['ltv']),
                     str(record['current_month_liquidation']), str(record['loan_age'])])

0	2015-04	2015-03	1426737600	-77	1	0
1	2015-04	2015-03	1426824000	16	1	0
2	2015-04	2015-03	1425873600	94	1	0
3	2015-04	2015-03	1426305600	-50	1	0
4	2015-04	2015-03	1425704400	-113	1	0
5	2015-04	2015-03	1427688000	-79	1	0
6	2015-04	2015-03	1426651200	16	1	0
7	2015-04	2015-03	1425618000	16	1	0
8	2015-04	2015-03	1426219200	16	1	0
9	2015-04	2015-03	1427428800	-25	1	0
10	2015-04	2015-03	1426219200	-72	1	0
11	2015-04	2015-03	1426564800	-106	1	0
12	2015-04	2015-03	1426219200	16	1	0
13	2015-04	2015-03	1426651200	-12	1	0
14	2015-04	2015-03	1427515200	-97	1	0
15	2015-04	2015-03	1427774400	-30	1	0
16	2015-04	2015-03	1426478400	-92	1	0
17	2015-04	2015-03	1426046400	-6	1	0
18	2015-04	2015-03	1426305600	98	1	0
19	2015-04	2015-03	1426046400	-117	1	0
20	2015-04	2015-03	1427428800	17	1	0
21	2015-04	2015-03	1426651200	16	1	0
22	2015-04	2015-03	1427774400	0	1	0
23	2015-04	2015-03	1425704400	-78	1	0
24	2015-04	2015-03	1427515200	105	1	0
25	2015-04	2015-03	1426478400	16	1	0
26	2015-04	2015-03	1425618000	11

Loan origination date does have data after 2015.

In [39]:
def check_data(x, condition="{} > 0"):
    for i, record in enumerate(h5file.root.loans.where(condition.format(x))):
        if i > 5:
            break
        else:
            print "\t".join([str(i), 
                         '{:%Y-%m}'.format(datetime.datetime.fromtimestamp(record['as_of_date'])),
                         str(record['dsn']),
                         str(record[x]),
                         str(record['ltv']),
                         str(record['current_month_liquidation']), str(record['loan_age'])])

In [40]:
check_data('upb_at_issuance')


0	2013-10	1014996591	20400000	8	1	53
1	2013-10	1015000553	13100000	-28	1	53
2	2013-10	1015024906	11000000	117	1	53
3	2013-10	1015173086	21900000	-21	1	53
4	2013-10	1015350193	5100000	101	1	54
5	2013-10	1015354318	7600000	46	1	53


In [41]:
check_data('loan_gross_margin')

0	2013-10	1010393116	2250	0	1	27
1	2013-10	1010401690	2250	0	0	0
2	2013-10	1010408228	2250	0	1	27
3	2013-10	1010410434	2000	40	1	27
4	2013-10	1010411999	2250	-78	1	27
5	2013-10	1010414466	2000	-79	1	27


You can have ARMs in GNM II loans!!

In [42]:
check_data('combined_ltv')

In [43]:
check_data('total_debt_expense_ratio')

0	2013-10	1500603755	48	28	1	85
1	2013-10	1500603756	4	-28	1	84
2	2013-10	1500603761	28	28	1	79
3	2013-10	1500603763	48	-11	1	77
4	2013-10	1500603764	92	46	1	76
5	2013-10	1500607617	36	-29	0	0


In [44]:
check_data('first_time')

0	2013-10	1023358115	1	0	1	-66
1	2013-10	1023181306	1	0	1	0
2	2013-10	1022642554	1	0	1	-78
3	2013-10	1022759447	1	0	1	-79
4	2013-10	1023377002	1	74	1	-82
5	2013-10	1022797607	1	87	1	-106


In [45]:
check_data('origination_type')

0	2013-10	1023586727	2	-22	1	33
1	2013-10	1500603755	2	28	1	85
2	2013-10	1500603756	2	-28	1	84
3	2013-10	1500603757	2	38	1	83
4	2013-10	1500603758	2	-30	0	0
5	2013-10	1500603759	2	11	0	0


In [46]:
check_data('removal_reason')

0	2013-10	1023598348	1	16	0	0
1	2013-10	1023596926	1	28	0	0
2	2013-10	1023595628	1	-112	0	0
3	2013-10	1023595629	1	-16	0	0
4	2013-10	1023596338	1	16	0	0
5	2013-10	1023597087	1	-16	0	0


So not that much. nearly everything is used.